In [ ]:
! git clone https://github.com/MinaGabriel/fuse-qa.git
%cd /content/fuse-qa
! export HF_TOKEN=
! pip install -e . 


Cloning into 'fuse-qa'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 40 (delta 10), reused 34 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 609.97 KiB | 13.55 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/fuse-qa
Obtaining file:///content/fuse-qa
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fuseqa (pyproject.toml) ... done
  Created wheel for fuseqa: filename=fuseqa-0.1.0-0.editable-py3-none-any.whl size=3454 sha256=0378f2c21b59a712fbf86674e06e37bef60ac6d3f0b954d164ab9b90a45030e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-c90672w3/wheels/a8/5d/ed/eaf3d3d6bc81274f10724114d2f643b34283e350e2beed7b2d
Successfully built fuseqa


In [2]:

import sys
import os
import torch
import tqdm
import json
from fuseqa import *
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from transformers.utils import logging
logging.set_verbosity_error()
from datetime import datetime

now = datetime.now().strftime("%Y-%m-%d %H:%M")
# runr> export HF_TOKEN=your_token_here
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
#MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
#MODEL_NAME = "google/gemma-2-2b-it" 
MODEL_NAME = "openai/gpt-oss-20b"
# Types: 
#   - PARAMETRIC:
#   - FUSEQA:
#   - FUSEQA-SRE:  
RUN_TYPE = "FUSEQA"
USE_CONTEXT = RUN_TYPE in ("FUSEQA", "FUSEQA-SRE")
print(hf_model_to_filename(MODEL_NAME + "-" + RUN_TYPE))
print("GPUs:", torch.cuda.device_count())

token = os.getenv("HF_TOKEN")
if token:
    login(token=token)
    print("Logged in to Hugging Face.")
else:
    print("HF_TOKEN not set.")



openai_gpt-oss-20b-FUSEQA_20260219-1615
GPUs: 1
HF_TOKEN not set.


In [3]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    #torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()
print("Model device:", next(model.parameters()).device)

if hasattr(model, "hf_device_map"):
    print("Device map:", model.hf_device_map)
else:
    print("Single-device load (no map needed)")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/411 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

Model device: cuda:0
Single-device load (no map needed)


In [4]:
# Loading the dataset
from datasets import load_dataset
# ds = load_dataset("MinaGabriel/popqa-with-retrieval-20")["test"].select(range(125))
ds = load_dataset("MinaGabriel/popqa-with-retrieval-20")["test"]
len(ds)

TOP_K = 3

README.md:   0%|          | 0.00/895 [00:00<?, ?B/s]

data/test-00000-of-00003.parquet:   0%|          | 0.00/174M [00:00<?, ?B/s]

data/test-00001-of-00003.parquet:   0%|          | 0.00/205M [00:00<?, ?B/s]

data/test-00002-of-00003.parquet:   0%|          | 0.00/217M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/14267 [00:00<?, ? examples/s]

In [6]:
from contextlib import nullcontext

import time

def run_popqa_eval(
    model,
    tokenizer,
    ds,
    write_outputs=False,
    groups=("ALL", "LONG-TAIL", "INFREQUENT", "FREQUENT"),
):
    start_time = time.time()  #START TIMER

    counts  = {g: 0 for g in groups}
    em_hits = {g: 0 for g in groups}

    def update_metrics(tier, em):
        counts["ALL"] += 1
        em_hits["ALL"] += em

        if tier in counts:
            counts[tier] += 1
            em_hits[tier] += em

    def current_scores():
        return {
            "ALL_EM":     safe_div(em_hits["ALL"],        counts["ALL"]),
            "Long_Tail":  safe_div(em_hits["LONG-TAIL"],  counts["LONG-TAIL"]),
            "Infrequent": safe_div(em_hits["INFREQUENT"], counts["INFREQUENT"]),
            "Frequent":   safe_div(em_hits["FREQUENT"],   counts["FREQUENT"]),
        }

    file_name = hf_model_to_filename(MODEL_NAME + "-" + RUN_TYPE)
    outfile   = file_name + ".jsonl"

    context_manager = open(outfile, "w", encoding="utf-8", buffering=1) if write_outputs else nullcontext()

    progress_bar = tqdm.tqdm(
        total=len(ds),
        desc="Generating + Evaluating",
        dynamic_ncols=True,
    )

    model_device = next(model.parameters()).device

    with context_manager as writer:
        for i in range(len(ds)):

            ex = {k: ds[k][i] for k in ds.column_names}

            q      = ex["question"]
            s_pop  = int(ex.get("s_pop", 0))
            tier   = tier_from_spop(s_pop)

            gold = parse_list(ex.get("possible_answers"))
            gold_norm_set = {norm(g) for g in gold if norm(g)}

            retrieved = ex.get("retrieved_docs") or []
            context   = build_context(retrieved, k=TOP_K) if USE_CONTEXT else ""

            pred = ask_llm_generate(
                model,
                tokenizer,
                q,
                context,
                use_context=USE_CONTEXT,
                device=model_device
            )
            print(f"\nQ: {q}\nContext: {context}\nPred: {pred}\nGold: {gold}\n")
            pred_norm = norm(pred)
            em = int(pred_norm in gold_norm_set) if gold_norm_set else 0

            update_metrics(tier, em)

            if write_outputs:
                record = {
                    "i": i,
                    "s_pop": s_pop,
                    "tier": tier,
                    "question": q,
                    "gold": gold,
                    "pred": pred,
                    "em": em,
                }
                write_record(writer, record)

            progress_bar.update(1)

            if i % 10 == 0:
                progress_bar.set_postfix({k: f"{v:.4f}" for k, v in current_scores().items()})

    progress_bar.close()

    total_time = time.time() - start_time  #END TIMER

    return counts, em_hits, file_name, total_time


# RUN
counts, em_hits, file_name, total_time = run_popqa_eval(
    model,
    tokenizer,
    ds,
    write_outputs=False
)

generate_report(
    counts,
    em_hits,
    file_name,
    model_name=MODEL_NAME,
    run_type=RUN_TYPE,
    total_time=total_time,
)



Q: What is George Rankin's occupation?
Context: "George Rankin"
George Rankin Major General George James Rankin, (1 May 1887 – 28 December 1957) was an Australian soldier and politician. He served in both the House of Representatives and the Senate, representing the Country Party of Australia. Ran

"George Rankin"
was buried, and was survived by his wife. George Rankin Major General George James Rankin, (1 May 1887 – 28 December 1957) was an Australian soldier and politician. He served in both the House of Representatives and the Senate, repres

"George Rankin"
lieutenant of the Australian Imperial Force and posted to the 4th Light Horse Regiment. Rankin served in the 4th Light Horse Regiment and reached Gallipoli in May 1915, was wounded in July, became a captain in December, and a major in
Pred: commentaryWe need
Gold: ['politician', 'political leader', 'political figure', 'polit.', 'pol']




Q: What is John Mayne's occupation?
Context: "John Mayne"
John Mayne John Mayne (1759–1836) was a Scottish printer, journalist and poet born in Dumfries, South West Scotland. In 1780, his poem ""The Siller Gun"" appeared in its original form in ""Ruddiman's Magazine"", published by Walter Ruddi

"John D. Mayne"
John D. Mayne John Dawson Mayne QC (1828–1917) was a British lawyer and legal expert who served as acting Advocate-General of the Madras Presidency and a member of the Privy Council of the United Kingdom. He is remembered as the autho

"Richard J. Maynes"
Richard J. Maynes Richard John Maynes (born October 29, 1950) has been a general authority of The Church of Jesus Christ of Latter-day Saints (LDS Church) since 1997. He served as a member of the church's Presidency of the Seventy
Pred: commentaryWe need
Gold: ['journalist', 'journo', 'journalists']




Q: What is Henry Feilden's occupation?
Context: "Henry Wemyss Feilden"
Henry Wemyss Feilden Colonel Henry Wemyss Feilden, CB (6 October 1838 – 8 June 1921) was a British Army officer, Arctic explorer and naturalist. Feilden was the second son of Sir William Henry Feilden (1812−1879), 2nd Baronet o

"Henry Wemyss Feilden"
was appointed a Companion of the Order of the Bath (CB) for his services to Imperial Yeomanry in 1900. Feilden also collected information on the geology, flora and fauna of newly explored areas, and served as naturalist on Sir 

"Henry Wemyss Feilden"
Feilden married Julia, daughter of Judge David James McCord (1797–1855) of South Carolina. In 1880 Feilden settled in Wells-next-the-Sea, Norfolk. Feilden joined the Norfolk and Norwich Naturalists' Society in 1880 and became P
Pred: commentaryWe need
Gold: ['politician', 'political leader', 'political figure', 'polit.', 'pol']




Q: What is Kathy Saltzman's occupation?
Context: "Kathy Saltzman"
Kathy Saltzman Kathy L. Saltzman (born June 4, 1955) is a Minnesota politician and a former member of the Minnesota Senate who represented District 56, which includes portions of Washington County in the eastern Twin Cities metropoli

"Kathy Saltzman"
for Equity in Education (SEE), working with school districts and parents throughout Minnesota on education funding issues, and also worked as an outreach and marketing employee for the University of Minnesota Hospital and Clinics. In

"Charles E. Saltzman"
the war as part of the Allied occupation of Austria, returning to the U.S. and leaving the military in 1946. He remained a member of the Reserves until 1955, retiring with the rank of major general. Saltzman briefly returned to 
Pred: commentaryWe need
Gold: ['politician', 'political leader', 'political figure', 'polit.', 'pol']




Q: What is Eleanor Davis's occupation?
Context: "Eleanor Davis"
comic book storytelling classes at the University of Georgia. Davis currently lives and works in Athens, Georgia, with fellow cartoonist and husband, Drew Weing. Eleanor Davis Eleanor McCutcheon Davis (born January 16, 1983) is an Ame

"Eleanor Davis"
Eleanor Davis Eleanor McCutcheon Davis (born January 16, 1983) is an American cartoonist and illustrator who creates comic works and other art for both adolescent and adult audiences. Eleanor Davis was raised in Tucson, Arizona by com

"Eleanor Davis"
Françoise Mouly's Toon Books. The book won her an ALA Geisel Honor Award in 2009. ""The Secret Science Alliance and the Copycat Crook"", published by Bloomsbury Children's in 2009, was a collaborative book created with husband Drew We
Pred: commentaryWe need
Gold: ['cartoonist', 'graphic artist', 'animator', 'illustrator']



...


Q: What is Fred C. Newmeyer's occupation?
Context: "Fred C. Newmeyer"
Fred C. Newmeyer Fred C. Newmeyer (August 9, 1888 – April 24, 1967) was an American actor, film director and film producer. A native of Central City, Colorado, he is best known for directing a handful of films in the ""Our Gang"" s

"Fred C. Newmeyer"
tested poorly, and producer Hal Roach scrapped most of the footage and remade the short with Robert McGowan as the director. Newmeyer, after directing numerous other shorts at Roach, would return to the ""Our Gang"" series in 1936 

"Discarded Lovers"
whodunit suspects abound and include Irma’s husband, a jealous wife, a boy friend and an ex-husband. Discarded Lovers Discarded Lovers is a 1932 American Pre-Code mystery film directed by Fred C. Newmeyer. Discarded Lovers is a mur
Pred: commentaryWe need
Gold: ['film director', 'movie director', 'director', 'motion picture director', 'film producer', 'movie producer', 'producer', 'actor', 'actress', 'actors', 'actresses'


Q: What is Edward Corser's occupation?
Context: "Edward Corser"
Edward Corser Edward Bernard Cresset Corser (1852 – 31 July 1928) was an Australian politician. Born in Birmingham, England, he was educated at Worcester and migrated to Australia in 1864. He was a sugar worker and bank officer in Mar

"Edward Corser"
party, he became a Nationalist. He held the seat until his death in 1928; he was succeeded in a by-election by his son, Bernard Corser, representing the Country Party. Corser died in 1928 and was buried in Maryborough Cemetery. Edward

"Eskdale, Maryborough"
not yet finished. Edward Corser arrived in Brisbane in about 1863 and moved to Maryborough in the next year. Eskdale was named by the Corsers as a reminder of Eskdale in the Lake District of Northern England. Edward Bernard Cor
Pred: commentaryWe need
Gold: ['politician', 'political leader', 'political figure', 'polit.', 'pol']




Q: What is Bae Geu-rin's occupation?
Context: "Entertainer (2016 TV series)"
entertainment company, KTOP Entertainment. He is a cunning and self-centered person in work fields. At first he only wants to use Jo Ha-neul to save his company and promises him to make him a well-known singer, but afte

"Hold My Hand (TV series)"
Hold My Hand (TV series) Hold My Hand () is a 2013 South Korean morning soap opera starring Park Si-eun, Bae Geu-rin, Lee Jae-hwang, and Jin Tae-hyun. It premiered on October 7, 2013 on MBC, airing every Monday to Friday at

"You Are the Best!"
that Soon-shin's birth mother is the famous actress Song Mi-ryung (Lee Mi-sook). Middle daughter Lee Yoo-shin (Yoo In-na) has lived next door to Park Chan-woo (Go Joo-won) since childhood. Despite their constant squabbles, Chan-wo
Pred: commentaryWe need
Gold: ['actor', 'actress', 'actors', 'actresses']




Q: What is Javier Alva Orlandini's occupation?
Context: "Javier Alva Orlandini"
Javier Alva Orlandini Javier Alva Orlandini (born December 11, 1927) is a lawyer and a Peruvian politician. A prominent member of the political party Popular Action, he serves as the President of the party. He served as 2nd Vi

"Javier Alva Orlandini"
the Court for a 3-year term. He was a candidate for the Presidency of the Republic in 1985, but got 4% of the popular vote, being defeated by Alan García. He later served as President of Peru's constitutional tribunal. He has 

"Giuseppe Maria Orlandini"
Giuseppe Maria Orlandini Giuseppe Maria Orlandini (4 April 167624 October 1760) was an Italian baroque composer particularly known for his more than 40 operas and intermezzos. Highly regarded by music historians of his day 
Pred: commentaryWe need
Gold: ['politician', 'political leader', 'political figure', 'polit.', 'pol']



KeyboardInterrupt: 